<a href="https://colab.research.google.com/github/michellehkim280/project_voting/blob/main/Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
! git clone https://github.com/carascott/project_voting

fatal: destination path 'project_voting' already exists and is not an empty directory.


In [37]:
pip install mapclassify

In [38]:
# Importing Packages, Loading Data

import geopandas as gpd
import mapclassify
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
import zipfile
import os

# Load voting data
voting_va = pd.read_csv('/content/project_voting/data/voting_VA.csv')

In [47]:
# Cleaning county_data

def load_data(file_path):
    return pd.read_csv(file_path, encoding='ISO-8859-1')

def prepare_data(df):
    # Standardize FIPS codes
    fips_columns = [col for col in ['FIPS', 'county_fips'] if col in df.columns]
    for col in fips_columns:
        df[col] = df[col].astype(str).str.zfill(5)

    # Remove unnecessary columns
    columns_to_drop = [col for col in ['Unneeded1', 'Unneeded2'] if col in df.columns]
    df.drop(columns=columns_to_drop, inplace=True)

    # Fill numeric columns with the median
    for col in df.select_dtypes(include='number').columns:
        df[col].fillna(df[col].median(), inplace=True)

    # Fill categorical columns with the mode (most frequent value)
    for col in df.select_dtypes(include='object').columns:
        df[col] = df[col].astype(str)
        if not df[col].mode().empty:
            df[col].fillna(df[col].mode()[0], inplace=True)

    # Remove rows with missing FIPS codes if critical
    df.dropna(subset=fips_columns, inplace=True)

    # Remove duplicates
    df.drop_duplicates(inplace=True)

    return df

# Directory to save cleaned files
output_directory = '/content/project_voting/data/county_data/cleaned'
os.makedirs(output_directory, exist_ok=True)

# Paths to the county data files
files = [
    '/content/project_voting/data/county_data/0002_ds176_20105_county_E.csv',
    '/content/project_voting/data/county_data/0002_ds177_20105_county_E.csv',
    '/content/project_voting/data/county_data/0002_ds191_20125_county_E.csv',
    '/content/project_voting/data/county_data/0002_ds206_20145_county_E.csv',
    '/content/project_voting/data/county_data/0002_ds225_20165_county_E.csv',
    '/content/project_voting/data/county_data/0002_ds239_20185_county_E.csv',
    '/content/project_voting/data/county_data/0002_ds249_20205_county_E.csv',
    '/content/project_voting/data/county_data/0002_ds250_20205_county_E.csv'
]

# Load, clean, and save each file
for file_path in files:
    df = load_data(file_path)
    cleaned_df = prepare_data(df)
    cleaned_file_path = os.path.join(output_directory, os.path.basename(file_path).replace('.csv', '_cleaned.csv'))
    cleaned_df.to_csv(cleaned_file_path, index=False)

<ipython-input-47-753ea21db3a0>:4: DtypeWarning: Columns (3,4,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,

In [48]:
# Cleaning voting_va

# Standardize FIPS codes to ensure they are five characters long, zero-padded
voting_va['county_fips'] = voting_va['county_fips'].astype(str).str.zfill(5)

# Convert all column names to lowercase for consistency
voting_va.columns = voting_va.columns.str.lower()

# Remove any rows where critical information is missing
voting_va.dropna(subset=['county_fips', 'candidatevotes', 'totalvotes'], inplace=True)

# Convert vote counts to integers (in case they are not)
voting_va['candidatevotes'] = voting_va['candidatevotes'].astype(int)
voting_va['totalvotes'] = voting_va['totalvotes'].astype(int)

# Check for any duplicates and remove them
voting_va.drop_duplicates(inplace=True)

# Save cleaned data
voting_va.to_csv('/content/project_voting/data/cleaned_voting_va.csv', index=False)

In [49]:
cleaned_df.head()

,GISJOIN,YEAR,STUSAB,REGIONA,DIVISIONA,STATE,STATEA,COUNTY,COUNTYA,COUSUBA,...,AM8FE031,AM8FE032,AM8FE033,AM8FE034,AM8FE035,AM8FE036,AM8FE037,AM8FE038,AM8FE039,AM9SE001
0,GIS Join Match Code,Data File Year,State Postal Abbreviation,Region Code,Division Code,State Name,State Code,County Name,County Code,County Subdivision Code,...,Female: 35 to 64 years,Female: 35 to 64 years: With a disability,Female: 35 to 64 years: No disability,Female: 65 to 74 years,Female: 65 to 74 years: With a disability,Female: 65 to 74 years: No disability,Female: 75 years and over,Female: 75 years and over: With a disability,Female: 75 years and over: No disability,Gini Index
1,G0100010,2016-2020,AL,nan,nan,Alabama,01,Autauga County,001,nan,...,11323,2064,9259,2741,898,1843,1962,1255,707,0.4552
2,G0100030,2016-2020,AL,nan,nan,Alabama,01,Baldwin County,003,nan,...,44915,5851,39064,14209,2921,11288,9539,4370,5169,0.4566
3,G0100050,2016-2020,AL,nan,nan,Alabama,01,Barbour County,005,nan,...,4243,859,3384,1554,436,1118,1074,709,365,0.5047
4,G0100070,2016-2020,AL,nan,nan,Alabama,01,Bibb County,007,nan,...,4217,656,3561,1098,235,863,871,433,438,0.4500


In [56]:
cleaned_voting_va = pd.read_csv('/content/project_voting/data/cleaned_voting_va.csv')
cleaned_voting_va.head()

,unnamed: 0,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
0,11161,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,AL GORE,DEMOCRAT,5092,11925,20220315,TOTAL
1,11162,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,6352,11925,20220315,TOTAL
2,11163,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,RALPH NADER,GREEN,220,11925,20220315,TOTAL
3,11164,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,OTHER,OTHER,261,11925,20220315,TOTAL
4,11165,2000,VIRGINIA,VA,ALBEMARLE,51003,US PRESIDENT,AL GORE,DEMOCRAT,16255,36846,20220315,TOTAL
